In [ ]:
%matplotlib inline

import numpy as np

#import BurstCube

import matplotlib as mpl
import matplotlib.pyplot as plt

import astropy.units as u
from astropy.time import Time

from gbm.data.phaii import TTE
from gbm.binning.unbinned import bin_by_time

## Methods and classes

In [ ]:
class Hit(object):
    """
    Single interaction with a SiPM
    
    Attributes
    ----------
        time      Time of the hits
        detector  Detector ID
        energy    Deposited energy
    """

    def __init__(self, detector, time, energy):

        self.time = time
        self.detector = detector
        self.energy = energy
        
    def __str__(self):
        return "Detector: {}   Time: {:.6f}   Energy: {:.2e}".format(self.detector, self.time, self.energy)
    
    def __lt__(self, other):
        '''
        Less than overload for time sorting
        '''
        return self.time < other.time

In [ ]:
from gbm.data.phaii import TTE
from gbm.data.primitives import EventList

class CosimaSim(object):
    '''
    Contains info associated with a Cosima output file (.sim)
    
    Attributes
    ----------
        tstart    Start of simulated time
        tstop     Stop of simulated time
        nthrown   Total number of simulated particles
        ntrigger  Total number of triggered events
        hits       Collection of all hits (Hit object array)
    
    '''
    
    def __init__(self):
        
        self.tstart = None
        self.tstop = None
        self.nthrown = None
        self.hits = []
    
    @classmethod
    def open(cls, filename, detector_dict, time_shift = 0*u.second):
    
        '''
        Parameters
        ----------
        filename: str
            Path to .sim file. Will be closed on destruction
    
        detector_dict: dict
            Convert from a string of x,y location to a given detector ID
            Use x and y value exactly as they appear in the input file
    
        time_shift: time Quantity, _optional_, default: 0s
            Apply a time shift to the timing in the file
    
        Return
        ------
            obj: CosimaSim object
    
        Warnings
        --------
        Concatenating files with NF and IN keywords is not supported
    
        '''
    
        obj = cls()
    
        file = open(filename)
    
        in_events = False
        while True:
        
            line = file.readline()
        
            if not line:
                break    
            
            key = line[0:2] # =\n for empty lines
            
            if in_events:
                
                if key == 'EN':
                    in_events = False
                    continue
                    
                # Time of hit
                if key == "TI":
                    time = float(line.split()[1]) * u.second + time_shift

                # Interaction with sensitive detectors
                if key != "HT":
                    continue
                    
                # Format
                x,y,energy = np.array(line.split(';'))[[1,2,4]]

                detector = detector_dict[x.strip()+','+y.strip()] 
                
                energy = float(energy)*u.keV
                
                # Add
                obj.hits = np.append(obj.hits, Hit(detector, time, energy))
        
            if key == 'TB':
                obj.tstart = float(line.split()[1])*u.second + time_shift
                
            elif key == 'SE':
                in_events = True
            
            elif key == 'TE':
                obj.tstop = float(line.split()[1])*u.second + time_shift
            
            elif key == 'TS':
                obj.nthrown = int(line.split()[1])
            
            elif key == 'NF' or key == 'IN':
                raise InputError("NF/IN keys found in {} not supported".format(filename))
    
        file.close()
    
        assert obj.tstart is not None, "TB key not found in {}".format(filename)
        assert obj.tstop is not None, "TE key not found in {}".format(filename)
        assert obj.nthrown is not None, "TS key not found in {}".format(filename)
        
        return obj

    def merge(self, sim2, shift = 0*u.second):
        
        '''
        Combine CosimaSim object with a shift
        
        Parameters
        ----------
            sim2: CosimaSim
                  Hits to be merged
            
            shift: time Quantity, _optional_, default: 0s
                Apply a time shift to obj2
            
        '''
        
        self.tstart = min(self.tstart, sim2.tstart + shift)
        self.tstop = max(self.tstop, sim2.tstop + shift)
        self.nthrown += sim2.nthrown
        
        for hit in sim2.hits:
            hit.time = hit.time + shift
            self.hits = np.append(self.hits, hit)
            
        hits_sort = np.argsort(self.hits)
        self.hits = [self.hits[n] for n in hits_sort]
        
    def to_TTE(self, energy_channel_edges, *args, **kwargs):
        
        '''
        Convert the hits into a mocked Time-Tagged Event object
        
        Parameters
        ----------
            energy_channel_edges: array
                Energy channel definition for discreatization
                
            Additional *args, **kwargs are passed to TTE.from_data()
                
        Return:
        -------
            tte: TTE object
        
        '''
        
        times = []
        energy_channels = []
    
        energy_channel_edges = [e.to(u.keV).value for e in energy_channel_edges]
        nchannels = len(energy_channel_edges)-1
    
        for hit in self.hits:
            times += [hit.time.to(u.second).value]
            
            energy_channel = np.digitize(hit.energy.to(u.keV).value, energy_channel_edges) - 1
            energy_channel = min(nchannels-1, max(0, energy_channel)) #Overflow/underflow
            
            energy_channels += [energy_channel]
        
        evtlist = EventList.from_lists(times_list = times, 
                                       pha_list = energy_channels, 
                                       chan_lo = energy_channel_edges[:-1],
                                       chan_hi = energy_channel_edges[1:])

        tte = TTE.from_data(data = evtlist, *args, **kwargs)
        
        return tte

        

## .sim to TTE

### Create object

In [ ]:
from gbm.binning.unbinned import bin_by_time

import matplotlib.pyplot as plt
from gbm.plot.lightcurve import Lightcurve

detector_dict = {"4.88500,4.88500":'q0', 
                 "-4.88500,4.88500":'q1', 
                 "-4.88500,-4.88500":'q2', 
                 "4.88500,-4.88500":'q3'}

t0 = 599529605.000 * u.second

sim = CosimaSim.open("sim/signal.inc1.id1.sim", detector_dict)
bkg = CosimaSim.open("sim/bkg.inc1.id1.sim", detector_dict)

sim.merge(bkg, shift=-5*u.second)

n_energy_bins = 128
energy_edges = [e*u.keV for e in np.geomspace(4.5,2000,n_energy_bins+1)]

tte = sim.to_TTE(energy_edges, trigtime = t0.to(u.second).value)

### IO

In [ ]:
tte.write("./")

In [ ]:
tte = TTE.open("glg_tte_all_bn200101000_v00.fit")

## Analisis

### Lightcurve

In [ ]:
phaii = tte.to_phaii(bin_by_time, .2, time_ref=0.0)

lcplot = Lightcurve(data=phaii.to_lightcurve())
plt.show()

### Spectrum

In [ ]:
from gbm.plot.spectrum import Spectrum

fig,ax = plt.subplots(dpi=150)

spectrum = tte.to_spectrum(time_range=(0.0, 2.0))

specplot = Spectrum(data=spectrum, axis=ax)

E = np.geomspace(30,200,100)

ax.plot(E, 2.5*(E/100)**-1.5, label=r"$\sim E^{-1.5}$")

ax.legend()

plt.show()